In [31]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.metrics import roc_auc_score


In [2]:
train = pd.read_csv('../data/train.csv')
students = pd.read_csv('../data/Student.csv')
internship = pd.read_csv('../data/Internship.csv')
test = pd.read_csv('../data/test.csv')

C:\Users\Abhishek\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
X = train[['Internship_ID', 'Is_Part_Time']].astype(np.float)
X.loc[:, 'hefty_internship'] = train.Expected_Stipend.map(lambda x: int(x == '10K+'))
X.loc[:, 'is_IHDF_location'] = train.Preferred_location.map(lambda x: int(x == 'IHDF'))

y = train.Is_Shortlisted

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=44)  

In [79]:
clf = RandomForestClassifier(n_estimators=75, max_depth=15, min_samples_split=5, n_jobs=-1)
pipeline = Pipeline([('clf', clf)])

In [80]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=15, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=75, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [81]:
predsTrain = pipeline.predict_proba(X_train)[:, 1]
predsTest = pipeline.predict_proba(X_test)[:, 1]

In [82]:
print 'AUC score on training set %f ' %(roc_auc_score(y_train, predsTrain))
print 'AUC score on test set %f ' %(roc_auc_score(y_test, predsTest))

AUC score on training set 0.829074 
AUC score on test set 0.782364 


In [83]:
pipeline.fit(X, y)

Pipeline(steps=[('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=15, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=75, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [85]:
test_subset = test[['Internship_ID','Is_Part_Time']].astype(np.float)
test_subset.loc[:, 'hefty_internship'] = test.Expected_Stipend.map(lambda x: int(x == '10K+'))
test_subset.loc[:, 'is_IHDF_location'] = test.Preferred_location.map(lambda x: int(x == 'IHDF'))

In [86]:
predictions = pipeline.predict_proba(test_subset)[:, 1]

In [87]:
submission = pd.read_csv('../data/submission.csv')
submission['Internship_ID'] = test.Internship_ID
submission['Student_ID'] = test.Student_ID
submission['Is_Shortlisted'] = predictions

In [88]:
submission.to_csv('../submissions/preferred_location_rf.csv', index=False)